In [2]:
import torch
from torch import nn
from lucent.optvis import render
from lucent.modelzoo import inceptionv1
import lucent
import torchvision.models as models
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class RedirectedReLU(torch.autograd.Function):
    """
    A workaround when there is no gradient flow from an initial random input
    See https://github.com/tensorflow/lucid/blob/master/lucid/misc/redirected_relu_grad.py
    Note: this means that the gradient is technically "wrong"
    TODO: the original Lucid library has a more sophisticated way of doing this
    """
    @staticmethod
    def forward(ctx, input_tensor):
        ctx.save_for_backward(input_tensor)
        return input_tensor.clamp(min=0)
    @staticmethod
    def backward(ctx, grad_output):
        input_tensor, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input_tensor < 0] = grad_input[input_tensor < 0] * 1e-1
        return grad_input


class RedirectedReluLayer(nn.Module):
    def forward(self, tensor):
        return RedirectedReLU.apply(tensor)

def redirect_relu(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            print("replacing relu", child_name)
            setattr(model, child_name, RedirectedReluLayer())
        else:
            redirect_relu(child)

In [4]:
def load_model(name):
    m=getattr(models, name)(pretrained=True)
    redirect_relu(m)
    m.to(device).eval()
    return m

resnet50 = load_model("resnet50")
# inceptionv2 = load_model("inception_v3")
# alexnet = load_model('alexnet')

replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu
replacing relu relu


In [6]:
print([n for n, x in resnet50.named_modules()])
# print(lucent.modelzoo.util.get_model_layers(resnet50))
# render.render_vis(resnet50, "avgpool:7")
render.render_vis(resnet50, "layer1:10")
# render.render_vis(resnet50, "layer3:11")

['', 'conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer1.0', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.conv3', 'layer1.0.bn3', 'layer1.0.relu', 'layer1.0.downsample', 'layer1.0.downsample.0', 'layer1.0.downsample.1', 'layer1.1', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.conv3', 'layer1.1.bn3', 'layer1.1.relu', 'layer1.2', 'layer1.2.conv1', 'layer1.2.bn1', 'layer1.2.conv2', 'layer1.2.bn2', 'layer1.2.conv3', 'layer1.2.bn3', 'layer1.2.relu', 'layer2', 'layer2.0', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.conv3', 'layer2.0.bn3', 'layer2.0.relu', 'layer2.0.downsample', 'layer2.0.downsample.0', 'layer2.0.downsample.1', 'layer2.1', 'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.conv2', 'layer2.1.bn2', 'layer2.1.conv3', 'layer2.1.bn3', 'layer2.1.relu', 'layer2.2', 'layer2.2.conv1', 'layer2.2.bn1', 'layer2.2.conv2', 'layer2.2.bn2', 'layer2.2.conv3', 'layer2.2.bn3', 'layer2.2.relu', 'layer2.

  0%|                            | 0/512 [00:00<?, ?it/s]c:\python39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████████████| 512/512 [02:41<00:00,  3.16it/s]


[array([[[[0.75847334, 0.6493587 , 0.99756026],
          [0.73796564, 0.98415935, 0.9990571 ],
          [0.7422461 , 0.97065926, 0.99800247],
          ...,
          [0.7007602 , 0.7670824 , 0.992747  ],
          [0.77368456, 0.95976704, 0.99760276],
          [0.64313143, 0.9850007 , 0.99870276]],
 
         [[0.71362203, 0.98937654, 0.99920505],
          [0.72293895, 0.97802645, 0.9988881 ],
          [0.7034192 , 0.92753994, 0.9978492 ],
          ...,
          [0.7246725 , 0.9441732 , 0.99614304],
          [0.76266646, 0.9728378 , 0.99844474],
          [0.8069213 , 0.6102967 , 0.99666226]],
 
         [[0.7479386 , 0.96247303, 0.9985769 ],
          [0.85322803, 0.62751216, 0.9972633 ],
          [0.67304426, 0.90440834, 0.9963257 ],
          ...,
          [0.76329106, 0.8243588 , 0.98917973],
          [0.6617194 , 0.93424016, 0.9950393 ],
          [0.718394  , 0.9241797 , 0.9974446 ]],
 
         ...,
 
         [[0.7902537 , 0.8440971 , 0.98394275],
          [0.74015